In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import pandas as pd
import numpy as np
import matplotlib
from IPython.display import *

import AssembleData

In [3]:

# Build Dataframe 
#MAL_df, all_genre  = AssembleData.read_files_all(verbose=True)
#MAL_df = MAL_df.drop_duplicates('id')

In [4]:
import pickle

#MAL_df = MAL_df.drop_duplicates('id')
#with open('augmented_1998_2015_data.pkl', 'w') as f:
#    pickle.dump(MAL_df, f)
#    pickle.dump(all_genre, f)

with open('augmented_1998_2015_data.pkl', 'r') as f:
    MAL_df = pickle.load(f)
    all_genre = pickle.load(f)

In [5]:
def only_genre(g):
    return MAL_df[MAL_df[g] == 1]

In [6]:
def get_std(entry):
    #print entry
    scores = []
    scores.append([10] * entry["score_10_votes"])
    scores.append([9] * entry["score_9_votes"])
    scores.append([8] * entry["score_8_votes"])
    scores.append([7] * entry["score_7_votes"])
    scores.append([6] * entry["score_6_votes"])
    scores.append([5] * entry["score_5_votes"])
    scores.append([4] * entry["score_4_votes"])
    scores.append([3] * entry["score_3_votes"])
    scores.append([2] * entry["score_2_votes"])
    scores.append([1] * entry["score_1_votes"])

    scores = [val for sublist in scores for val in sublist]

    #print scores
    return np.std(scores)

def get_df_std(df):
    scores = []
    for idx, entry in df.iterrows():
        scores.append([10] * entry["score_10_votes"])
        scores.append([9] * entry["score_9_votes"])
        scores.append([8] * entry["score_8_votes"])
        scores.append([7] * entry["score_7_votes"])
        scores.append([6] * entry["score_6_votes"])
        scores.append([5] * entry["score_5_votes"])
        scores.append([4] * entry["score_4_votes"])
        scores.append([3] * entry["score_3_votes"])
        scores.append([2] * entry["score_2_votes"])
        scores.append([1] * entry["score_1_votes"])

    scores = [val for sublist in scores for val in sublist]

    #print scores
    return np.std(scores)


In [7]:
MAL_df["std"]  = MAL_df.apply(lambda x: get_std(x), axis = 1)
MAL_df['length'] = MAL_df['duration'] * MAL_df['episodes']
MAL_df['studios'] = MAL_df['studios'].apply(lambda s: eval(s))
MAL_df['aired_start'] = pd.to_datetime(MAL_df['aired_start'])

#print MAL_df.head()

In [44]:
sig_length = MAL_df[MAL_df['length'] > 30]

stat_df = sig_length[sig_length['score_users'] > 20000]
stat_df = stat_df.reset_index()

In [55]:
print stat_df.iloc[0]['related_ids']

#stat_df = stat_df.set_index(['id'])
#print stat_df.index

for idx in eval(stat_df.iloc[0]['related_ids']):
    if (len(stat_df[stat_df['id']==idx]['title']) > 0):
        print stat_df[stat_df['id']==idx]['title']
        #DO MERING HERE

#stat_df.iloc[0]['title']

[5, 17205, 4037]
64    Cowboy Bebop: Tengoku no Tobira
Name: title, dtype: object
